# 🧬 Two-Stage Microbiome Disease Classification Pipeline

## 🎯 Pipeline Overview

This notebook implements a sophisticated two-stage classification system:

**Stage 1:** Binary Classification (Healthy vs Diseased)
- Distinguishes between healthy and diseased samples
- Outputs probability of disease

**Stage 2:** Multi-Class Disease Classification
- Predicts specific disease type for diseased samples
- Computes risk scores across all disease classes

**Final Output:** Combined risk assessment with probabilities for all classes including Healthy

## 📦 Cell 1: Install and Import Dependencies

In [1]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
import os
import json
import pickle
from pathlib import Path
from collections import Counter

# Scikit-learn imports
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, roc_curve, auc,
    precision_recall_fscore_support,
    accuracy_score, f1_score
)
from sklearn.utils import class_weight

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn.functional as F

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


🖥️  Using device: cuda
GPU: Tesla T4


## 📂 Cell 2: Configuration and File Paths

In [3]:
# Configuration
CONFIG = {
    # File paths
    'healthy_file': '/content/drive/MyDrive/Gut Sean/disease_data/Healthy.csv',
    'disease_dir': '/content/drive/MyDrive/Gut Sean/combined_csv_files/',
    'output_dir': '/content/drive/MyDrive/Gut Sean/outputs/',

    # Preprocessing
    'log_transform': True,
    'log_constant': 1e-6,
    'min_variance_threshold': 1e-10,
    'drop_columns': ['Run_ID', 'SampleID', 'Sample_ID', 'ID'],

    # Feature selection
    'n_features': 500,
    'feature_selection_method': 'f_classif',

    # Train/val/test split
    'test_size': 0.15,
    'val_size': 0.15,
    'random_state': 42,

    # ========== UPDATED: Class imbalance handling ==========
    'use_weighted_sampler': False,      # CHANGED: Disable weighted sampler
    'undersample_healthy': True,
    'undersample_ratio': 1.0,            # CHANGED: 1:1 ratio instead of 0.5:1
    'use_class_weights': True,           # Keep weighted loss

    # ========== UPDATED: Model architecture ==========
    'hidden_dims': [1024, 512, 256],    # CHANGED: Wider network
    'dropout': 0.3,
    'use_batch_norm': True,

    # ========== UPDATED: Training parameters ==========
    'batch_size': 64,                    # CHANGED: Larger batches
    'learning_rate': 0.001,
    'weight_decay': 5e-5,                # CHANGED: Reduced weight decay
    'num_epochs': 100,
    'patience': 15,                      # CHANGED: More patience

    # ========== NEW: Learning rate scheduling ==========
    'use_lr_scheduler': True,
    'scheduler_type': 'cosine',          # 'cosine' or 'plateau'
    'lr_warmup_epochs': 5,

    # ========== UPDATED: Stage 2 threshold ==========
    'disease_threshold_low': 0.3,        # NEW: Soft thresholding
    'disease_threshold_high': 0.7,       # NEW: Soft thresholding

    # ========== NEW: Focal loss parameters ==========
    'use_focal_loss': True,
    'focal_alpha': 0.25,
    'focal_gamma': 2.0,
}

## 🔧 Cell 3: Data Loading Function

In [4]:
def load_all_data(healthy_file, disease_dir, verbose=True):
    """
    Load Healthy.csv and all disease CSV files dynamically.

    Returns:
        all_data: List of (dataframe, label) tuples
        disease_names: List of disease names
    """
    all_data = []
    disease_names = ['Healthy']

    # Load Healthy samples
    if verbose:
        print("\n📊 Loading data files...")
        print("=" * 60)

    try:
        df_healthy = pd.read_csv(healthy_file)
        all_data.append((df_healthy, 'Healthy'))
        if verbose:
            print(f"✓ Healthy: {len(df_healthy)} samples, {len(df_healthy.columns)} features")
    except Exception as e:
        print(f"❌ Error loading Healthy.csv: {e}")
        return None, None

    # Load disease files
    disease_files = glob.glob(os.path.join(disease_dir, '*.csv'))

    if not disease_files:
        print(f"⚠️  No disease CSV files found in {disease_dir}")
        return None, None

    for file_path in sorted(disease_files):
        disease_name = Path(file_path).stem

        try:
            df_disease = pd.read_csv(file_path)
            all_data.append((df_disease, disease_name))
            disease_names.append(disease_name)

            if verbose:
                print(f"✓ {disease_name}: {len(df_disease)} lines, {len(df_disease.columns)} features")
        except Exception as e:
            print(f"⚠️  Error loading {disease_name}: {e}")

    if verbose:
        print("=" * 60)
        print(f"\n✅ Loaded {len(all_data)} datasets ({len(disease_names)} classes)")

    return all_data, disease_names


# Load data
all_data, disease_names = load_all_data(
    CONFIG['healthy_file'],
    CONFIG['disease_dir']
)

if all_data is None:
    raise ValueError("Failed to load data. Please check file paths.")


📊 Loading data files...
✓ Healthy: 1320693 samples, 5 features
✓ COVID19: 119774 lines, 5 features
✓ Colorectal_Neoplasms: 93665 lines, 5 features
✓ Crohn_s: 97457 lines, 5 features
✓ Diabetes: 46962 lines, 5 features
✓ IBS: 22118 lines, 5 features
✓ KidneyFailure: 43111 lines, 5 features
✓ Parkinsons: 67049 lines, 5 features
✓ Ulcerative_Colitis: 64838 lines, 5 features
✓ cysticfibrosis: 31888 lines, 5 features
✓ nafld: 52297 lines, 5 features

✅ Loaded 11 datasets (11 classes)


In [5]:
## 🔄 Cell 3.5: Pivot Long Format to Wide Format (INSERT THIS AFTER CELL 3)

def pivot_long_to_wide(all_data, verbose=True):
    """
    Convert microbiome data from long format to wide format.

    Long format: Each row = one taxon abundance in one sample
    Wide format: Each row = one sample with all taxon abundances as columns

    Args:
        all_data: List of (dataframe, label) tuples in long format
        verbose: Print progress information

    Returns:
        processed_data: List of (dataframe, label) tuples in wide format
    """
    if verbose:
        print("\n🔄 Pivoting data from long to wide format...")
        print("=" * 60)

    processed_data = []

    for i, (df, label) in enumerate(all_data):
        # Make a copy to avoid modifying original
        df_copy = df.copy()

        # Normalize column names (lowercase, replace spaces with underscores)
        df_copy.columns = df_copy.columns.str.lower().str.replace(' ', '_')

        if verbose:
            print(f"\n📊 Processing {label}:")
            print(f"   Original shape: {df.shape}")
            print(f"   Columns: {list(df_copy.columns)}")

        # Verify required columns exist
        required_cols = ['run_id', 'ncbi_taxon_id', 'relative_abundance']
        missing_cols = [col for col in required_cols if col not in df_copy.columns]

        if missing_cols:
            print(f"   ⚠️  WARNING: Missing columns {missing_cols}. Skipping {label}.")
            continue

        # Get unique counts before pivot
        n_samples = df_copy['run_id'].nunique()
        n_taxa = df_copy['ncbi_taxon_id'].nunique()

        if verbose:
            print(f"   Unique samples (run_id): {n_samples}")
            print(f"   Unique taxa (ncbi_taxon_id): {n_taxa}")

        try:
            # Pivot the data
            df_wide = df_copy.pivot_table(
                index='run_id',
                columns='ncbi_taxon_id',
                values='relative_abundance',
                fill_value=0.0,
                aggfunc='first'  # In case of duplicates, take first value
            )

            # Rename columns to be more descriptive
            df_wide.columns = [f'taxon_{col}' for col in df_wide.columns]

            # Reset index to make run_id a column
            df_wide = df_wide.reset_index()

            if verbose:
                print(f"   ✓ Pivoted shape: {df_wide.shape}")
                print(f"   ✓ Samples: {len(df_wide)}, Features: {len(df_wide.columns) - 1}")

            # Add to processed data
            processed_data.append((df_wide, label))

        except Exception as e:
            print(f"   ❌ Error pivoting {label}: {e}")
            continue

    if verbose:
        print("\n" + "=" * 60)
        print(f"✅ Pivoting complete!")
        print(f"   Processed {len(processed_data)} / {len(all_data)} datasets")
        print(f"   Ready for preprocess_data() function")

    return processed_data


# Run the pivot operation
processed_data = pivot_long_to_wide(all_data, verbose=True)

# Replace all_data with processed_data
all_data = processed_data

# Summary
print("\n📋 Summary:")
print("=" * 60)
total_samples = 0
total_features = 0

for df, label in all_data:
    n_samples = len(df)
    n_features = len(df.columns) - 1  # Exclude run_id
    total_samples += n_samples
    total_features = max(total_features, n_features)
    print(f"{label:25s}: {n_samples:6d} samples × {n_features:5d} features")

print("=" * 60)
print(f"Total: {total_samples} samples, up to {total_features} unique taxa")
print(f"\n✅ Data is now in wide format and ready for preprocessing!")


🔄 Pivoting data from long to wide format...

📊 Processing Healthy:
   Original shape: (1320693, 5)
   Columns: ['run_id', 'ncbi_taxon_id', 'relative_abundance', 'taxon_rank_level', 'scientific_name']
   Unique samples (run_id): 12236
   Unique taxa (ncbi_taxon_id): 2179
   ✓ Pivoted shape: (12236, 2180)
   ✓ Samples: 12236, Features: 2179

📊 Processing COVID19:
   Original shape: (119774, 5)
   Columns: ['run_id', 'ncbi_taxon_id', 'relative_abundance', 'taxon_rank_level', 'scientific_name']
   Unique samples (run_id): 1813
   Unique taxa (ncbi_taxon_id): 1273
   ✓ Pivoted shape: (1813, 1274)
   ✓ Samples: 1813, Features: 1273

📊 Processing Colorectal_Neoplasms:
   Original shape: (93665, 5)
   Columns: ['run_id', 'ncbi_taxon_id', 'relative_abundance', 'taxon_rank_level', 'scientific_name']
   Unique samples (run_id): 793
   Unique taxa (ncbi_taxon_id): 1377
   ✓ Pivoted shape: (793, 1378)
   ✓ Samples: 793, Features: 1377

📊 Processing Crohn_s:
   Original shape: (97457, 5)
   Columns

## 🧹 Cell 4: Preprocessing Pipeline

In [6]:
## 🔧 Cell 4: Updated Preprocessing (REPLACE CELL 4 WITH THIS)

def preprocess_data(all_data, disease_names, config, verbose=True):
    """
    Complete preprocessing pipeline for WIDE FORMAT data:
    1. Align features across all datasets
    2. Drop identifier columns (run_id, etc.)
    3. Remove zero-variance features
    4. Log-transform abundances
    5. Combine into single DataFrame

    Returns:
        X: Feature matrix (numpy array)
        y: Labels (numpy array)
        feature_names: List of feature column names
        label_encoder: Fitted LabelEncoder
    """
    if verbose:
        print("\n🔧 Starting preprocessing pipeline...")
        print("=" * 60)

    # Step 1: Collect all unique features
    all_features = set()
    for df, _ in all_data:
        all_features.update(df.columns)

    # Remove identifier columns (including run_id from pivot)
    drop_cols = set(config['drop_columns'])
    drop_cols.add('run_id')  # Add run_id from pivot operation

    features_to_keep = sorted(all_features - drop_cols)

    if verbose:
        print(f"✓ Total unique features found: {len(all_features)}")
        print(f"✓ Dropping identifier columns: {drop_cols}")
        print(f"✓ Features after filtering: {len(features_to_keep)}")

    # Step 2: Align all datasets to common feature set
    aligned_data = []
    labels = []

    for df, label in all_data:
        # Create aligned dataframe with float type
        df_aligned = pd.DataFrame(0.0, index=df.index, columns=features_to_keep)

        # Fill in existing features
        for col in features_to_keep:
            if col in df.columns:
                df_aligned[col] = df[col].values

        aligned_data.append(df_aligned)
        labels.extend([label] * len(df))

    # Step 3: Combine all data
    X_df = pd.concat(aligned_data, axis=0, ignore_index=True)

    if verbose:
        print(f"\n✓ Combined dataset shape: {X_df.shape}")
        print(f"✓ This represents {len(X_df)} samples (run_ids)")

    # Step 3.5: Drop any remaining non-numeric columns (safety check)
    numeric_cols = []
    non_numeric_cols = []

    for col in X_df.columns:
        try:
            pd.to_numeric(X_df[col], errors='raise')
            numeric_cols.append(col)
        except (ValueError, TypeError):
            non_numeric_cols.append(col)

    if non_numeric_cols:
        if verbose:
            print(f"\n⚠️  Found {len(non_numeric_cols)} non-numeric columns:")
            for col in non_numeric_cols[:5]:
                sample_value = X_df[col].iloc[0] if len(X_df) > 0 else "N/A"
                print(f"   - {col}: (example: {sample_value})")
            if len(non_numeric_cols) > 5:
                print(f"   ... and {len(non_numeric_cols) - 5} more")

        X_df = X_df[numeric_cols]

        if verbose:
            print(f"✓ Dropped {len(non_numeric_cols)} non-numeric columns")

    # Ensure all numeric
    X_df = X_df.astype(float)

    if verbose:
        print(f"✓ Remaining numeric features: {X_df.shape[1]}")

    # Step 4: Remove zero/low variance features
    variances = X_df.var()
    low_var_cols = variances[variances < config['min_variance_threshold']].index
    X_df = X_df.drop(columns=low_var_cols)

    if verbose:
        print(f"✓ Removed {len(low_var_cols)} low-variance features")
        print(f"✓ Remaining features: {X_df.shape[1]}")

    # Step 5: Log transformation
    if config['log_transform']:
        X_df = np.log(X_df + config['log_constant'])
        if verbose:
            print(f"✓ Applied log transformation: log(x + {config['log_constant']})")

    # Step 6: Convert to numpy arrays
    X = X_df.values
    feature_names = X_df.columns.tolist()

    # Encode labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    if verbose:
        print("\n✓ Label encoding:")
        for idx, name in enumerate(label_encoder.classes_):
            print(f"   {idx}: {name} ({np.sum(y == idx)} samples)")
        print("=" * 60)
        print(f"\n✅ Preprocessing complete!")
        print(f"   Final shape: {X.shape}")
        print(f"   Samples (actual run_ids): {len(y)}")
        print(f"   Features (microbial taxa): {len(feature_names)}")
        print(f"   Classes: {len(np.unique(y))}")

    return X, y, feature_names, label_encoder


# Run preprocessing
X_raw, y, feature_names, label_encoder = preprocess_data(
    all_data,
    disease_names,
    CONFIG
)

# Print class distribution
print("\n📊 Class Distribution:")
print("=" * 60)
class_counts = Counter(y)
for class_idx in sorted(class_counts.keys()):
    class_name = label_encoder.inverse_transform([class_idx])[0]
    count = class_counts[class_idx]
    percentage = 100 * count / len(y)
    print(f"{class_name:25s}: {count:6d} samples ({percentage:6.2f}%)")


🔧 Starting preprocessing pipeline...
✓ Total unique features found: 2565
✓ Dropping identifier columns: {'run_id', 'Sample_ID', 'SampleID', 'Run_ID', 'ID'}
✓ Features after filtering: 2564

✓ Combined dataset shape: (19708, 2564)
✓ This represents 19708 samples (run_ids)
✓ Remaining numeric features: 2564
✓ Removed 186 low-variance features
✓ Remaining features: 2378
✓ Applied log transformation: log(x + 1e-06)

✓ Label encoding:
   0: COVID19 (1813 samples)
   1: Colorectal_Neoplasms (793 samples)
   2: Crohn_s (1425 samples)
   3: Diabetes (326 samples)
   4: Healthy (12236 samples)
   5: IBS (379 samples)
   6: KidneyFailure (347 samples)
   7: Parkinsons (549 samples)
   8: Ulcerative_Colitis (859 samples)
   9: cysticfibrosis (488 samples)
   10: nafld (493 samples)

✅ Preprocessing complete!
   Final shape: (19708, 2378)
   Samples (actual run_ids): 19708
   Features (microbial taxa): 2378
   Classes: 11

📊 Class Distribution:
COVID19                  :   1813 samples (  9.20%)


## 🎯 Cell 5: Feature Selection

In [7]:
def select_top_features(X, y, feature_names, n_features, method='f_classif', verbose=True):
    """
    Perform feature selection using ANOVA F-test or Mutual Information.

    Args:
        X: Feature matrix
        y: Labels
        feature_names: List of feature names
        n_features: Number of top features to select
        method: 'f_classif' or 'mutual_info'

    Returns:
        X_selected: Selected features
        selector: Fitted SelectKBest object
        selected_feature_names: Names of selected features
    """
    if verbose:
        print("\n🔬 Feature Selection Pipeline")
        print("=" * 60)
        print(f"Method: {method}")
        print(f"Selecting top {n_features} features from {X.shape[1]}")

    # Choose scoring function
    if method == 'f_classif':
        score_func = f_classif
    elif method == 'mutual_info':
        score_func = mutual_info_classif
    else:
        raise ValueError(f"Unknown method: {method}")

    # Perform feature selection
    n_features_to_select = min(n_features, X.shape[1])
    selector = SelectKBest(score_func=score_func, k=n_features_to_select)
    X_selected = selector.fit_transform(X, y)

    # Get selected feature names
    selected_indices = selector.get_support(indices=True)
    selected_feature_names = [feature_names[i] for i in selected_indices]

    # Get feature scores
    scores = selector.scores_[selected_indices]

    if verbose:
        print(f"\n✓ Selected {len(selected_feature_names)} features")
        print(f"✓ New feature matrix shape: {X_selected.shape}")
        print(f"\n📈 Top 10 features by score:")
        top_10_idx = np.argsort(scores)[-10:][::-1]
        for i, idx in enumerate(top_10_idx, 1):
            print(f"   {i:2d}. {selected_feature_names[idx]:50s} (score: {scores[idx]:.2f})")
        print("=" * 60)

    return X_selected, selector, selected_feature_names


# Perform feature selection
X_selected, feature_selector, selected_features = select_top_features(
    X_raw,
    y,
    feature_names,
    CONFIG['n_features'],
    CONFIG['feature_selection_method']
)


🔬 Feature Selection Pipeline
Method: f_classif
Selecting top 500 features from 2378

✓ Selected 500 features
✓ New feature matrix shape: (19708, 500)

📈 Top 10 features by score:
    1. taxon_28131                                        (score: 421.69)
    2. taxon_853                                          (score: 312.52)
    3. taxon_341694                                       (score: 299.86)
    4. taxon_745368                                       (score: 297.64)
    5. taxon_1352                                         (score: 295.28)
    6. taxon_2981769                                      (score: 272.89)
    7. taxon_39950                                        (score: 268.11)
    8. taxon_351091                                       (score: 262.49)
    9. taxon_1628085                                      (score: 261.45)
   10. taxon_2885350                                      (score: 244.85)


## ⚖️ Cell 6: Train/Val/Test Split with Stratification

In [8]:
# Cell 6.5: Undersample THEN Split (Combined)

def undersample_majority_class(X, y, healthy_idx=4, ratio=2.0, random_state=42):
    """
    Undersample the Healthy class to reduce severe imbalance.
    Returns both undersampled data AND the indices that were kept.
    """
    print("\n⚖️ Undersampling Healthy Class to Reduce Imbalance")
    print("=" * 60)

    # Split into healthy and diseased
    healthy_mask = (y == healthy_idx)
    diseased_mask = ~healthy_mask

    healthy_indices = np.where(healthy_mask)[0]
    diseased_indices = np.where(diseased_mask)[0]

    print(f"Before - Healthy: {len(healthy_indices):,}, Diseased: {len(diseased_indices):,}")
    print(f"         Ratio: {len(healthy_indices)/len(diseased_indices):.2f}:1")

    # Calculate target healthy samples
    n_diseased = len(diseased_indices)
    n_healthy_target = int(n_diseased * ratio)

    # Randomly sample healthy INDICES
    np.random.seed(random_state)
    if n_healthy_target < len(healthy_indices):
        sampled_healthy_indices = np.random.choice(healthy_indices, n_healthy_target, replace=False)
    else:
        sampled_healthy_indices = healthy_indices

    # Combine all kept indices
    kept_indices = np.concatenate([sampled_healthy_indices, diseased_indices])

    # Shuffle
    np.random.shuffle(kept_indices)

    # Extract data
    X_balanced = X[kept_indices]
    y_balanced = y[kept_indices]

    print(f"After  - Healthy: {n_healthy_target:,}, Diseased: {n_diseased:,}")
    print(f"         New Ratio: {n_healthy_target/n_diseased:.2f}:1")
    print(f"✅ Removed {len(healthy_indices) - n_healthy_target:,} healthy samples")
    print("=" * 60)

    return X_balanced, y_balanced, kept_indices


def create_train_val_test_split(X, y, test_size=0.15, val_size=0.15, random_state=42, verbose=True):
    """
    Create stratified train/validation/test splits.

    Returns:
        Dictionary containing train/val/test splits
    """
    if verbose:
        print("\n📊 Creating Train/Val/Test Split")
        print("=" * 60)

    # First split: train+val vs test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y,
        test_size=test_size,
        stratify=y,
        random_state=random_state
    )

    # Second split: train vs val
    val_size_adjusted = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=val_size_adjusted,
        stratify=y_temp,
        random_state=random_state
    )

    splits = {
        'X_train': X_train,
        'y_train': y_train,
        'X_val': X_val,
        'y_val': y_val,
        'X_test': X_test,
        'y_test': y_test
    }

    if verbose:
        print(f"Train set: {X_train.shape[0]} samples ({100*len(X_train)/len(X):.1f}%)")
        print(f"Val set:   {X_val.shape[0]} samples ({100*len(X_val)/len(X):.1f}%)")
        print(f"Test set:  {X_test.shape[0]} samples ({100*len(X_test)/len(X):.1f}%)")

        print("\n📊 Class distribution per split:")
        for split_name, split_y in [('Train', y_train), ('Val', y_val), ('Test', y_test)]:
            print(f"\n{split_name}:")
            unique, counts = np.unique(split_y, return_counts=True)
            for u, c in zip(unique, counts):
                print(f"   Class {u}: {c:4d} samples ({100*c/len(split_y):5.1f}%)")
        print("=" * 60)

    return splits


# ============================================================
# STEP 1: UNDERSAMPLE (if enabled)
# ============================================================
if CONFIG['undersample_healthy']:
    X_selected, y, kept_indices = undersample_majority_class(
        X_selected,
        y,
        healthy_idx=4,
        ratio=CONFIG['undersample_ratio'],
        random_state=CONFIG['random_state']
    )

    print(f"\n✅ Dataset after undersampling: {X_selected.shape}")
    print(f"✅ Kept {len(kept_indices)} samples")

    # CRITICAL: Also undersample X_raw using the same indices
    X_raw = X_raw[kept_indices]
    print(f"✅ X_raw also undersampled: {X_raw.shape}")
else:
    print("\n⚠️ Undersampling disabled in CONFIG")

# ============================================================
# STEP 2: SPLIT DATA (after undersampling!)
# ============================================================
print("\n🔄 Creating train/val/test splits from undersampled data...")
data_splits = create_train_val_test_split(
    X_selected,
    y,
    test_size=CONFIG['test_size'],
    val_size=CONFIG['val_size'],
    random_state=CONFIG['random_state'],
    verbose=True
)
print("✅ Splits created with balanced data")



⚖️ Undersampling Healthy Class to Reduce Imbalance
Before - Healthy: 12,236, Diseased: 7,472
         Ratio: 1.64:1
After  - Healthy: 7,472, Diseased: 7,472
         New Ratio: 1.00:1
✅ Removed 4,764 healthy samples

✅ Dataset after undersampling: (14944, 500)
✅ Kept 14944 samples
✅ X_raw also undersampled: (14944, 2378)

🔄 Creating train/val/test splits from undersampled data...

📊 Creating Train/Val/Test Split
Train set: 10460 samples (70.0%)
Val set:   2242 samples (15.0%)
Test set:  2242 samples (15.0%)

📊 Class distribution per split:

Train:
   Class 0: 1269 samples ( 12.1%)
   Class 1:  555 samples (  5.3%)
   Class 2:  997 samples (  9.5%)
   Class 3:  228 samples (  2.2%)
   Class 4: 5230 samples ( 50.0%)
   Class 5:  265 samples (  2.5%)
   Class 6:  243 samples (  2.3%)
   Class 7:  385 samples (  3.7%)
   Class 8:  601 samples (  5.7%)
   Class 9:  342 samples (  3.3%)
   Class 10:  345 samples (  3.3%)

Val:
   Class 0:  272 samples ( 12.1%)
   Class 1:  119 samples (  5.

## 🔄 Cell 7: Normalization (StandardScaler)

In [9]:
# Fit scaler on training data only
scaler = StandardScaler()
data_splits['X_train'] = scaler.fit_transform(data_splits['X_train'])
data_splits['X_val'] = scaler.transform(data_splits['X_val'])
data_splits['X_test'] = scaler.transform(data_splits['X_test'])

print("\n✅ Applied StandardScaler normalization")
print(f"   Training mean: {data_splits['X_train'].mean():.6f}")
print(f"   Training std:  {data_splits['X_train'].std():.6f}")


✅ Applied StandardScaler normalization
   Training mean: 0.000000
   Training std:  1.000000


In [10]:
# Cell 8.5: Focal Loss Implementation

class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance.
    Focuses training on hard examples.

    FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    """
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

print("✅ Focal Loss class defined")

✅ Focal Loss class defined


## ⚖️ Cell 8: Handle Class Imbalance

In [11]:
def compute_class_weights(y, verbose=True):
    """
    Compute class weights inversely proportional to class frequencies.
    """
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y),
        y=y
    )

    if verbose:
        print("\n⚖️  Class Weights (balanced):")
        print("=" * 60)
        for class_idx, weight in enumerate(weights):
            class_name = label_encoder.inverse_transform([class_idx])[0]
            print(f"   {class_name:25s}: {weight:.4f}")
        print("=" * 60)

    return torch.FloatTensor(weights)


def create_weighted_sampler(y, verbose=True):
    """
    Create WeightedRandomSampler for oversampling rare classes.
    """
    class_counts = np.bincount(y)
    weights = 1.0 / class_counts[y]
    sampler = WeightedRandomSampler(
        weights=weights,
        num_samples=len(weights),
        replacement=True
    )

    if verbose:
        print("\n✅ Created WeightedRandomSampler for balanced sampling")

    return sampler


# Compute class weights for all classes (multi-class)
class_weights_multiclass = compute_class_weights(data_splits['y_train'])

# Compute class weights for binary (Healthy vs Diseased)
y_train_binary = (data_splits['y_train'] !=4).astype(int)  # 0=Healthy, 1=Diseased
class_weights_binary = compute_class_weights(y_train_binary, verbose=False)

print("\n⚖️  Binary Class Weights (Healthy vs Diseased):")
print(f"   Healthy:  {class_weights_binary[0]:.4f}")
print(f"   Diseased: {class_weights_binary[1]:.4f}")

# Optionally create weighted sampler
if CONFIG['use_weighted_sampler']:
    train_sampler = create_weighted_sampler(data_splits['y_train'])
else:
    train_sampler = None


⚖️  Class Weights (balanced):
   COVID19                  : 0.7493
   Colorectal_Neoplasms     : 1.7133
   Crohn_s                  : 0.9538
   Diabetes                 : 4.1707
   Healthy                  : 0.1818
   IBS                      : 3.5883
   KidneyFailure            : 3.9132
   Parkinsons               : 2.4699
   Ulcerative_Colitis       : 1.5822
   cysticfibrosis           : 2.7804
   nafld                    : 2.7563

⚖️  Binary Class Weights (Healthy vs Diseased):
   Healthy:  1.0000
   Diseased: 1.0000


## 📦 Cell 9: PyTorch Dataset and DataLoader

In [12]:
class MicrobiomeDataset(Dataset):
    """Custom PyTorch Dataset for microbiome data."""

    def __init__(self, X, y, binary_labels=False):
        """
        Args:
            X: Feature matrix (numpy array)
            y: Labels (numpy array)
            binary_labels: If True, convert to binary (0=Healthy, 1=Diseased)
        """
        self.X = torch.FloatTensor(X)

        if binary_labels:

            healthy_idx = 4
            y_binary = (y != healthy_idx).astype(int)
            self.y = torch.LongTensor(y_binary)
        else:
            self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def create_dataloaders(splits, batch_size, sampler=None, binary=False):
    """
    Create PyTorch DataLoaders for train/val/test sets.
    """
    train_dataset = MicrobiomeDataset(
        splits['X_train'],
        splits['y_train'],
        binary_labels=binary
    )
    val_dataset = MicrobiomeDataset(
        splits['X_val'],
        splits['y_val'],
        binary_labels=binary
    )
    test_dataset = MicrobiomeDataset(
        splits['X_test'],
        splits['y_test'],
        binary_labels=binary
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=(sampler is None),
        sampler=sampler
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    return train_loader, val_loader, test_loader


print("\n✅ Dataset and DataLoader classes defined")


✅ Dataset and DataLoader classes defined


## 🧠 Cell 10: Model Architecture (Shared MLP)

In [13]:
class DiseaseClassifier(nn.Module):
    """
    Multi-layer Perceptron (MLP) for disease classification.
    Can be used for both binary and multi-class classification.
    """

    def __init__(self, input_dim, num_classes, hidden_dims=[512, 256, 128],
                 dropout=0.3, use_batch_norm=True):
        super(DiseaseClassifier, self).__init__()

        layers = []
        prev_dim = input_dim

        # Hidden layers
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))

            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_dim))

            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))

            prev_dim = hidden_dim

        # Output layer
        layers.append(nn.Linear(prev_dim, num_classes))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


# Print model architecture
input_dim = data_splits['X_train'].shape[1]
num_classes = len(np.unique(y))

print("\n🧠 Model Architecture:")
print("=" * 60)
print(f"Input dimension: {input_dim}")
print(f"Hidden layers: {CONFIG['hidden_dims']}")
print(f"Dropout: {CONFIG['dropout']}")
print(f"Batch normalization: {CONFIG['use_batch_norm']}")
print(f"\nBinary output: 2 classes (Healthy vs Diseased)")
print(f"Multi-class output: {num_classes} classes")
print("=" * 60)


🧠 Model Architecture:
Input dimension: 500
Hidden layers: [1024, 512, 256]
Dropout: 0.3
Batch normalization: True

Binary output: 2 classes (Healthy vs Diseased)
Multi-class output: 11 classes


## 🏋️ Cell 11: Training Function

In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs, patience, device, scheduler=None,
                warmup_epochs=0, model_name="Model"):
    """
    Train a PyTorch model with early stopping and optional LR scheduling.

    New features:
    - Learning rate warmup
    - LR scheduler support
    - Better progress reporting
    """
    print(f"\n🏋️  Training {model_name}...")
    print("=" * 60)

    model = model.to(device)
    best_val_loss = float('inf')
    best_val_f1 = 0.0  # NEW: Also track best F1
    best_model_state = None
    patience_counter = 0

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'learning_rates': []  # NEW: Track LR
    }

    # Get base learning rate for warmup
    base_lr = optimizer.param_groups[0]['lr']

    for epoch in range(num_epochs):
        # ========== NEW: Learning rate warmup ==========
        if epoch < warmup_epochs:
            warmup_lr = base_lr * (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = warmup_lr

        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            # Backward pass
            loss.backward()

            # NEW: Gradient clipping (prevents exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Statistics
            train_loss += loss.item() * batch_X.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == batch_y).sum().item()
            train_total += batch_y.size(0)

        train_loss = train_loss / train_total
        train_acc = train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)

                val_loss += loss.item() * batch_X.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == batch_y).sum().item()
                val_total += batch_y.size(0)

                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(batch_y.cpu().numpy())

        val_loss = val_loss / val_total
        val_acc = val_correct / val_total

        # NEW: Compute validation F1
        val_f1 = f1_score(val_labels, val_preds, average='macro')

        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']

        # Store history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['learning_rates'].append(current_lr)

        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1:3d}/{num_epochs}] "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f} | "
                  f"LR: {current_lr:.6f}")

        # ========== NEW: LR Scheduler step ==========
        if scheduler is not None:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()

        # ========== UPDATED: Early stopping on F1 score ==========
        if val_f1 > best_val_f1:
            best_val_loss = val_loss
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"\n⏹️  Early stopping at epoch {epoch+1}")
            break

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    print("=" * 60)
    print(f"✅ Training complete!")
    print(f"   Best validation loss: {best_val_loss:.4f}")
    print(f"   Best validation F1: {best_val_f1:.4f}")

    return model, history


print("\n✅ Updated training function defined")


✅ Updated training function defined


## 📊 Cell 12: Evaluation Function

In [15]:
def evaluate_model(model, test_loader, device, class_names, stage_name="Model"):
    """
    Comprehensive evaluation of a trained model.

    Returns:
        results: Dictionary containing all metrics
    """
    print(f"\n📊 Evaluating {stage_name}...")
    print("=" * 60)

    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.numpy())
            all_probs.extend(probs.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)

    # Per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        all_labels, all_preds, average=None, zero_division=0
    )

    # Macro/Micro averages
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    micro_f1 = f1_score(all_labels, all_preds, average='micro')

    # AUROC
    if len(class_names) > 2:
        # Multi-class AUROC
        auroc_macro = roc_auc_score(
            all_labels, all_probs,
            average='macro',
            multi_class='ovr'
        )
        auroc_micro = roc_auc_score(
            all_labels, all_probs,
            average='micro',
            multi_class='ovr'
        )
    else:
        # Binary AUROC
        auroc_macro = roc_auc_score(all_labels, all_probs[:, 1])
        auroc_micro = auroc_macro

    # Print results
    print(f"\n🎯 Overall Metrics:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Macro F1:  {macro_f1:.4f}")
    print(f"   Micro F1:  {micro_f1:.4f}")
    print(f"   AUROC (Macro): {auroc_macro:.4f}")
    print(f"   AUROC (Micro): {auroc_micro:.4f}")

    print(f"\n📊 Per-Class Metrics:")
    print(f"{'Class':<20s} {'Precision':>10s} {'Recall':>10s} {'F1':>10s} {'Support':>10s}")
    print("-" * 60)
    for i, class_name in enumerate(class_names):
        print(f"{class_name:<20s} {precision[i]:>10.4f} {recall[i]:>10.4f} "
              f"{f1[i]:>10.4f} {support[i]:>10d}")
    print("=" * 60)

    results = {
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs,
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'micro_f1': micro_f1,
        'auroc_macro': auroc_macro,
        'auroc_micro': auroc_micro,
        'per_class_precision': precision,
        'per_class_recall': recall,
        'per_class_f1': f1,
        'per_class_support': support
    }

    return results


print("\n✅ Evaluation function defined")


✅ Evaluation function defined


## 🩺 Cell 14: STAGE 1 - Binary Classifier (Healthy vs Diseased)

In [16]:
print("\n" + "="*80)
print("🩺 STAGE 1: BINARY CLASSIFICATION (Healthy vs Diseased)")
print("="*80)

# Create binary dataloaders
train_loader_binary, val_loader_binary, test_loader_binary = create_dataloaders(
    data_splits,
    CONFIG['batch_size'],
    sampler=None,  # No weighted sampler
    binary=True
)

print(f"\n✓ Created binary dataloaders")
print(f"   Batch size: {CONFIG['batch_size']}")

# Initialize Stage 1 model
model_stage1 = DiseaseClassifier(
    input_dim=input_dim,
    num_classes=2,
    hidden_dims=CONFIG['hidden_dims'],
    dropout=CONFIG['dropout'],
    use_batch_norm=CONFIG['use_batch_norm']
)

# ========== UPDATED: Loss function ==========
if CONFIG['use_focal_loss']:
    criterion_stage1 = FocalLoss(
        alpha=CONFIG['focal_alpha'],
        gamma=CONFIG['focal_gamma']
    )
    print(f"✓ Using Focal Loss (alpha={CONFIG['focal_alpha']}, gamma={CONFIG['focal_gamma']})")
elif CONFIG['use_class_weights']:
    criterion_stage1 = nn.CrossEntropyLoss(weight=class_weights_binary.to(device))
    print(f"✓ Using weighted CrossEntropyLoss")
else:
    criterion_stage1 = nn.CrossEntropyLoss()
    print(f"✓ Using CrossEntropyLoss")

# ========== UPDATED: Optimizer (AdamW) ==========
optimizer_stage1 = optim.AdamW(  # Changed from Adam to AdamW
    model_stage1.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

print(f"✓ Optimizer: AdamW (lr={CONFIG['learning_rate']}, wd={CONFIG['weight_decay']})")

# ========== NEW: Learning Rate Scheduler ==========
if CONFIG['use_lr_scheduler']:
    if CONFIG['scheduler_type'] == 'cosine':
        scheduler_stage1 = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer_stage1,
            T_0=10,  # Restart every 10 epochs
            T_mult=2  # Double the restart interval each time
        )
        print(f"✓ Using CosineAnnealingWarmRestarts scheduler")
    elif CONFIG['scheduler_type'] == 'plateau':
        scheduler_stage1 = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer_stage1,
            mode='min',
            factor=0.5,
            patience=5,
            verbose=True
        )
        print(f"✓ Using ReduceLROnPlateau scheduler")
else:
    scheduler_stage1 = None
    print(f"✓ No LR scheduler")

# Train Stage 1
model_stage1, history_stage1 = train_model(
    model_stage1,
    train_loader_binary,
    val_loader_binary,
    criterion_stage1,
    optimizer_stage1,
    CONFIG['num_epochs'],
    CONFIG['patience'],
    device,
    scheduler=scheduler_stage1,
    warmup_epochs=CONFIG['lr_warmup_epochs'],
    model_name="Stage 1 (Binary)"
)

# Evaluate Stage 1
binary_class_names = ['Healthy', 'Diseased']
results_stage1 = evaluate_model(
    model_stage1,
    test_loader_binary,
    device,
    binary_class_names,
    stage_name="Stage 1 (Binary)"
)


🩺 STAGE 1: BINARY CLASSIFICATION (Healthy vs Diseased)

✓ Created binary dataloaders
   Batch size: 64
✓ Using Focal Loss (alpha=0.25, gamma=2.0)
✓ Optimizer: AdamW (lr=0.001, wd=5e-05)
✓ Using CosineAnnealingWarmRestarts scheduler

🏋️  Training Stage 1 (Binary)...
Epoch [  1/100] Train Loss: 0.0423 Acc: 0.6848 | Val Loss: 0.0299 Acc: 0.7797 F1: 0.7788 | LR: 0.000200
Epoch [  5/100] Train Loss: 0.0218 Acc: 0.8498 | Val Loss: 0.0281 Acc: 0.8162 F1: 0.8161 | LR: 0.001000
Epoch [ 10/100] Train Loss: 0.0065 Acc: 0.9627 | Val Loss: 0.0364 Acc: 0.8421 F1: 0.8420 | LR: 0.000024
Epoch [ 15/100] Train Loss: 0.0096 Acc: 0.9453 | Val Loss: 0.0349 Acc: 0.8354 F1: 0.8353 | LR: 0.000905
Epoch [ 20/100] Train Loss: 0.0048 Acc: 0.9738 | Val Loss: 0.0521 Acc: 0.8345 F1: 0.8345 | LR: 0.000578
Epoch [ 25/100] Train Loss: 0.0019 Acc: 0.9895 | Val Loss: 0.0648 Acc: 0.8390 F1: 0.8389 | LR: 0.000206

⏹️  Early stopping at epoch 25
✅ Training complete!
   Best validation loss: 0.0364
   Best validation F1: 0

## 🧫 Cell 16: STAGE 2 - Multi-Class Disease Classifier

In [17]:
print("\n" + "="*80)
print("🧫 STAGE 2: MULTI-CLASS DISEASE CLASSIFICATION")
print("="*80)

print("\n🔍 Filtering Training Data: DISEASED SAMPLES ONLY")
print("="*60)

# Filter out Healthy samples (class 4) from all splits
healthy_idx = 4

# Training set - diseased only
diseased_train_mask = (data_splits['y_train'] != healthy_idx)
X_train_diseased = data_splits['X_train'][diseased_train_mask]
y_train_diseased_original = data_splits['y_train'][diseased_train_mask]

# Validation set - diseased only
diseased_val_mask = (data_splits['y_val'] != healthy_idx)
X_val_diseased = data_splits['X_val'][diseased_val_mask]
y_val_diseased_original = data_splits['y_val'][diseased_val_mask]

# Test set - diseased only
diseased_test_mask = (data_splits['y_test'] != healthy_idx)
X_test_diseased = data_splits['X_test'][diseased_test_mask]
y_test_diseased_original = data_splits['y_test'][diseased_test_mask]

print(f"Original - Train: {len(data_splits['y_train'])}, Val: {len(data_splits['y_val'])}, Test: {len(data_splits['y_test'])}")
print(f"Diseased - Train: {len(X_train_diseased)}, Val: {len(X_val_diseased)}, Test: {len(X_test_diseased)}")

# Create mapping from original labels to Stage 2 labels (0-9 for 10 diseases)
original_disease_classes = [i for i in range(len(label_encoder.classes_)) if i != healthy_idx]
disease_idx_to_name = {}
disease_label_mapping = {}

for new_idx, original_idx in enumerate(original_disease_classes):
    disease_name = label_encoder.inverse_transform([original_idx])[0]
    disease_idx_to_name[new_idx] = disease_name
    disease_label_mapping[original_idx] = new_idx

print(f"\n📋 Disease Label Mapping (10 diseases, no Healthy):")
for new_idx, disease_name in disease_idx_to_name.items():
    print(f"   Stage 2 index {new_idx} → {disease_name}")
print("="*60)

# Remap labels for Stage 2
y_train_stage2 = np.array([disease_label_mapping[y] for y in y_train_diseased_original])
y_val_stage2 = np.array([disease_label_mapping[y] for y in y_val_diseased_original])
y_test_stage2 = np.array([disease_label_mapping[y] for y in y_test_diseased_original])

# Create class names for Stage 2 (10 diseases)
multiclass_names_stage2 = [disease_idx_to_name[i] for i in range(len(disease_idx_to_name))]

print(f"\n✅ Stage 2 will classify these {len(multiclass_names_stage2)} diseases:")
for i, name in enumerate(multiclass_names_stage2):
    print(f"   {i}: {name}")
print("="*60)

# Compute class weights for Stage 2 (10 diseases only)
class_weights_stage2 = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_stage2),
    y=y_train_stage2
)
class_weights_stage2 = torch.FloatTensor(class_weights_stage2)

print(f"\n⚖️ Class weights for Stage 2 (10 diseases):")
for i, weight in enumerate(class_weights_stage2):
    print(f"   {multiclass_names_stage2[i]:25s}: {weight:.4f}")

# Create datasets for Stage 2
train_dataset_stage2 = MicrobiomeDataset(X_train_diseased, y_train_stage2, binary_labels=False)
val_dataset_stage2 = MicrobiomeDataset(X_val_diseased, y_val_stage2, binary_labels=False)
test_dataset_stage2 = MicrobiomeDataset(X_test_diseased, y_test_stage2, binary_labels=False)

# Create dataloaders for Stage 2
train_loader_stage2 = DataLoader(
    train_dataset_stage2,
    batch_size=CONFIG['batch_size'],
    shuffle=True
)
val_loader_stage2 = DataLoader(
    val_dataset_stage2,
    batch_size=CONFIG['batch_size'],
    shuffle=False
)
test_loader_stage2 = DataLoader(
    test_dataset_stage2,
    batch_size=CONFIG['batch_size'],
    shuffle=False
)

print(f"\n✓ Created multi-class dataloaders (diseased samples only)")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Number of classes: {len(multiclass_names_stage2)}")

# Initialize Stage 2 model
model_stage2 = DiseaseClassifier(
    input_dim=input_dim,
    num_classes=len(multiclass_names_stage2),  # 10 diseases
    hidden_dims=CONFIG['hidden_dims'],
    dropout=CONFIG['dropout'],
    use_batch_norm=CONFIG['use_batch_norm']
)

# ========== UPDATED: Loss function with Focal Loss ==========
if CONFIG['use_focal_loss']:
    criterion_stage2 = FocalLoss(
        alpha=CONFIG['focal_alpha'],
        gamma=CONFIG['focal_gamma']
    )
    print(f"✓ Using Focal Loss for {len(multiclass_names_stage2)} disease classes")
elif CONFIG['use_class_weights']:
    criterion_stage2 = nn.CrossEntropyLoss(weight=class_weights_stage2.to(device))
    print(f"✓ Using weighted CrossEntropyLoss for {len(multiclass_names_stage2)} disease classes")
else:
    criterion_stage2 = nn.CrossEntropyLoss()
    print(f"✓ Using CrossEntropyLoss")

# ========== UPDATED: Optimizer (AdamW) ==========
optimizer_stage2 = optim.AdamW(
    model_stage2.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

print(f"✓ Optimizer: AdamW (lr={CONFIG['learning_rate']}, wd={CONFIG['weight_decay']})")

# ========== NEW: Learning Rate Scheduler ==========
if CONFIG['use_lr_scheduler']:
    if CONFIG['scheduler_type'] == 'cosine':
        scheduler_stage2 = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer_stage2,
            T_0=10,
            T_mult=2
        )
        print(f"✓ Using CosineAnnealingWarmRestarts scheduler")
    elif CONFIG['scheduler_type'] == 'plateau':
        scheduler_stage2 = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer_stage2,
            mode='min',
            factor=0.5,
            patience=5,
            verbose=True
        )
        print(f"✓ Using ReduceLROnPlateau scheduler")
else:
    scheduler_stage2 = None
    print(f"✓ No LR scheduler")

# Train Stage 2
model_stage2, history_stage2 = train_model(
    model_stage2,
    train_loader_stage2,
    val_loader_stage2,
    criterion_stage2,
    optimizer_stage2,
    CONFIG['num_epochs'],
    CONFIG['patience'],
    device,
    scheduler=scheduler_stage2,
    warmup_epochs=CONFIG['lr_warmup_epochs'],
    model_name="Stage 2 (Multi-Class)"
)

# Evaluate Stage 2
results_stage2 = evaluate_model(
    model_stage2,
    test_loader_stage2,
    device,
    multiclass_names_stage2,
    stage_name="Stage 2 (Multi-Class - Diseases Only)"
)


🧫 STAGE 2: MULTI-CLASS DISEASE CLASSIFICATION

🔍 Filtering Training Data: DISEASED SAMPLES ONLY
Original - Train: 10460, Val: 2242, Test: 2242
Diseased - Train: 5230, Val: 1121, Test: 1121

📋 Disease Label Mapping (10 diseases, no Healthy):
   Stage 2 index 0 → COVID19
   Stage 2 index 1 → Colorectal_Neoplasms
   Stage 2 index 2 → Crohn_s
   Stage 2 index 3 → Diabetes
   Stage 2 index 4 → IBS
   Stage 2 index 5 → KidneyFailure
   Stage 2 index 6 → Parkinsons
   Stage 2 index 7 → Ulcerative_Colitis
   Stage 2 index 8 → cysticfibrosis
   Stage 2 index 9 → nafld

✅ Stage 2 will classify these 10 diseases:
   0: COVID19
   1: Colorectal_Neoplasms
   2: Crohn_s
   3: Diabetes
   4: IBS
   5: KidneyFailure
   6: Parkinsons
   7: Ulcerative_Colitis
   8: cysticfibrosis
   9: nafld

⚖️ Class weights for Stage 2 (10 diseases):
   COVID19                  : 0.4121
   Colorectal_Neoplasms     : 0.9423
   Crohn_s                  : 0.5246
   Diabetes                 : 2.2939
   IBS               

## 🔗 Cell 18: Combined Two-Stage Inference Function

In [21]:
def predict_with_risk(sample, model_stage1, model_stage2, scaler,
                     feature_selector, label_encoder, disease_idx_to_name,
                     disease_label_mapping, device,
                     threshold_low=0.3, threshold_high=0.7):
    """
    Two-stage prediction with SOFT THRESHOLDING and uncertainty handling.

    Args:
        threshold_low: Below this, classify as Healthy
        threshold_high: Above this, classify as Diseased
        Between low and high: Mark as Uncertain

    Returns:
        Dictionary with prediction, confidence, and risk scores
    """
    model_stage1.eval()
    model_stage2.eval()

    # ========== FIXED: Correct preprocessing order ==========
    # 1. Feature selection first (2378 → 500 features)
    sample_selected = feature_selector.transform(sample.reshape(1, -1))
    # 2. Then scaling (500 → 500 features)
    sample_scaled = scaler.transform(sample_selected)
    # 3. Convert to tensor
    sample_tensor = torch.FloatTensor(sample_scaled).to(device)

    with torch.no_grad():
        # Stage 1: Healthy vs Diseased
        output_stage1 = model_stage1(sample_tensor)
        probs_stage1 = F.softmax(output_stage1, dim=1).cpu().numpy()[0]

        healthy_prob = probs_stage1[0]
        disease_prob = probs_stage1[1]

        # Stage 2: Multi-class probabilities
        output_stage2 = model_stage2(sample_tensor)
        probs_stage2 = F.softmax(output_stage2, dim=1).cpu().numpy()[0]

        # ========== Soft thresholding logic ==========
        if healthy_prob > threshold_high:
            # High confidence it's healthy
            predicted_name = "Healthy"
            confidence = "High"
        elif disease_prob > threshold_high:
            # High confidence it's diseased
            stage2_predicted_idx = np.argmax(probs_stage2)
            predicted_name = disease_idx_to_name[stage2_predicted_idx]
            confidence = "High"
        elif healthy_prob > threshold_low:
            # Leaning healthy but uncertain
            predicted_name = "Healthy"
            confidence = "Low"
        elif disease_prob > threshold_low:
            # Leaning diseased but uncertain
            stage2_predicted_idx = np.argmax(probs_stage2)
            predicted_name = disease_idx_to_name[stage2_predicted_idx]
            confidence = "Low"
        else:
            # Very uncertain - defer to highest Stage 2 probability
            stage2_predicted_idx = np.argmax(probs_stage2)
            predicted_name = disease_idx_to_name[stage2_predicted_idx]
            confidence = "Uncertain"

        # Compute risk scores
        healthy_idx = np.where(label_encoder.classes_ == "Healthy")[0][0]
        risk_scores = np.zeros(len(label_encoder.classes_))
        risk_scores[healthy_idx] = healthy_prob

        for stage2_idx in range(len(disease_idx_to_name)):
            disease_name = disease_idx_to_name[stage2_idx]
            original_idx = np.where(label_encoder.classes_ == disease_name)[0][0]
            risk_scores[original_idx] = disease_prob * probs_stage2[stage2_idx]

        risk_scores = risk_scores / risk_scores.sum()

    # Build results
    per_class_probs = {}
    for i, class_name in enumerate(label_encoder.classes_):
        per_class_probs[class_name] = float(risk_scores[i])

    results_dict = {
        "Predicted": predicted_name,
        "Confidence": confidence,
        "Healthy_Prob": float(healthy_prob),
        "Disease_Prob": float(disease_prob),
        "Risk_Weighted_Probabilities": per_class_probs
    }

    return results_dict


print("\n✅ Updated two-stage prediction function with soft thresholding")


✅ Updated two-stage prediction function with soft thresholding


## 🧪 Cell 19: Test Two-Stage Pipeline on Sample

In [22]:
print("\n🧪 Testing Two-Stage Pipeline")
print("="*60)

# Test on a few random samples from test set
num_samples_to_test = 5

# ✅ FIXED: X_raw and y are now aligned, so we can safely sample
test_indices = np.random.choice(len(X_raw), num_samples_to_test, replace=False)

for i, idx in enumerate(test_indices, 1):
    sample = X_raw[idx]
    true_label = y[idx]
    true_name = label_encoder.inverse_transform([true_label])[0]

    print(f"\n📋 Sample {i} (True Label: {true_name})")
    print("-" * 60)

    result = predict_with_risk(
        sample,
        model_stage1,
        model_stage2,
        scaler,
        feature_selector,
        label_encoder,
        disease_idx_to_name,
        disease_label_mapping,
        device,
        threshold_low=CONFIG['disease_threshold_low'],    # UPDATED
        threshold_high=CONFIG['disease_threshold_high']   # UPDATED
    )

    print(json.dumps(result, indent=2))

    # Check if prediction is correct
    correct = "✓" if result['Predicted'] == true_name else "✗"
    print(f"\n{correct} Prediction: {'Correct' if correct == '✓' else 'Incorrect'}")

    # NEW: Show confidence level
    print(f"   Confidence: {result['Confidence']}")

print("\n" + "="*60)


🧪 Testing Two-Stage Pipeline

📋 Sample 1 (True Label: Healthy)
------------------------------------------------------------
{
  "Predicted": "Healthy",
  "Confidence": "High",
  "Healthy_Prob": 0.9762320518493652,
  "Disease_Prob": 0.023767979815602303,
  "Risk_Weighted_Probabilities": {
    "COVID19": 0.0003340649020728591,
    "Colorectal_Neoplasms": 2.4794183563001198e-05,
    "Crohn_s": 0.013540181829919685,
    "Diabetes": 0.0003495772141066997,
    "Healthy": 0.9762320206493376,
    "IBS": 5.6702085209952863e-05,
    "KidneyFailure": 1.2303140072315416e-05,
    "Parkinsons": 0.00026910201135593097,
    "Ulcerative_Colitis": 0.009113701700466229,
    "cysticfibrosis": 4.345771091984356e-05,
    "nafld": 2.4094572975915713e-05
  }
}

✓ Prediction: Correct
   Confidence: High

📋 Sample 2 (True Label: COVID19)
------------------------------------------------------------
{
  "Predicted": "COVID19",
  "Confidence": "High",
  "Healthy_Prob": 0.014937133528292179,
  "Disease_Prob": 0.98

## 🎯 Cell 20: Evaluate Full Two-Stage Pipeline on Test Set

In [23]:
def evaluate_two_stage_pipeline(X_test, y_test, model_stage1, model_stage2,
                                scaler, feature_selector, label_encoder,
                                disease_idx_to_name, disease_label_mapping,
                                threshold_low, threshold_high, device):  # UPDATED
    """
    Evaluate the complete two-stage pipeline on test set.
    """
    print("\n🎯 Evaluating Two-Stage Pipeline on Test Set")
    print("="*60)

    all_predictions = []
    all_risk_scores = []
    all_confidences = []  # NEW: Track confidence

    for i in range(len(X_test)):
        result = predict_with_risk(
            X_test[i],
            model_stage1,
            model_stage2,
            scaler,
            feature_selector,
            label_encoder,
            disease_idx_to_name,
            disease_label_mapping,
            device,
            threshold_low=threshold_low,    # NEW
            threshold_high=threshold_high   # NEW
        )

        predicted_class = label_encoder.transform([result['Predicted']])[0]
        all_predictions.append(predicted_class)
        all_confidences.append(result['Confidence'])  # NEW

        risk_vector = [result['Risk_Weighted_Probabilities'][name]
                      for name in label_encoder.classes_]
        all_risk_scores.append(risk_vector)

    all_predictions = np.array(all_predictions)
    all_risk_scores = np.array(all_risk_scores)

    # Compute metrics
    accuracy = accuracy_score(y_test, all_predictions)
    macro_f1 = f1_score(y_test, all_predictions, average='macro')
    micro_f1 = f1_score(y_test, all_predictions, average='micro')

    # AUROC
    auroc_macro = roc_auc_score(y_test, all_risk_scores, average='macro', multi_class='ovr')
    auroc_micro = roc_auc_score(y_test, all_risk_scores, average='micro', multi_class='ovr')

    print(f"\n🎯 Two-Stage Pipeline Performance:")
    print(f"   Accuracy:      {accuracy:.4f}")
    print(f"   Macro F1:      {macro_f1:.4f}")
    print(f"   Micro F1:      {micro_f1:.4f}")
    print(f"   AUROC (Macro): {auroc_macro:.4f}")
    print(f"   AUROC (Micro): {auroc_micro:.4f}")

    # NEW: Report confidence distribution
    from collections import Counter
    conf_dist = Counter(all_confidences)
    print(f"\n📊 Confidence Distribution:")
    for conf_level in ['High', 'Low', 'Uncertain']:
        count = conf_dist.get(conf_level, 0)
        pct = 100 * count / len(all_confidences)
        print(f"   {conf_level:10s}: {count:4d} ({pct:5.1f}%)")

    print("="*60)

    results = {
        'predictions': all_predictions,
        'labels': y_test,
        'probabilities': all_risk_scores,
        'confidences': all_confidences,  # NEW
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'micro_f1': micro_f1,
        'auroc_macro': auroc_macro,
        'auroc_micro': auroc_micro
    }

    return results

# ✅ FIXED: X_raw and y are now aligned after undersampling!
# Create aligned splits - they're already aligned now
raw_splits = create_train_val_test_split(
    X_raw,  # Already undersampled and aligned with y
    y,      # Already undersampled
    test_size=CONFIG['test_size'],
    val_size=CONFIG['val_size'],
    random_state=CONFIG['random_state'],
    verbose=False
)

# Evaluate complete pipeline
# Evaluate complete pipeline with soft thresholding
pipeline_results = evaluate_two_stage_pipeline(
    raw_splits['X_test'],
    raw_splits['y_test'],
    model_stage1,
    model_stage2,
    scaler,
    feature_selector,
    label_encoder,
    disease_idx_to_name,
    disease_label_mapping,
    CONFIG['disease_threshold_low'],   # NEW: Use low threshold
    CONFIG['disease_threshold_high'],  # NEW: Use high threshold
    device
)


🎯 Evaluating Two-Stage Pipeline on Test Set

🎯 Two-Stage Pipeline Performance:
   Accuracy:      0.7435
   Macro F1:      0.6295
   Micro F1:      0.7435
   AUROC (Macro): 0.9527
   AUROC (Micro): 0.9705

📊 Confidence Distribution:
   High      : 1911 ( 85.2%)
   Low       :  331 ( 14.8%)
   Uncertain :    0 (  0.0%)


## 💾 Cell 22: Save Models and Artifacts

In [ ]:
print("\n💾 Saving models and artifacts...")
print("="*60)

# Create output directory
output_dir = CONFIG['output_dir']
os.makedirs(output_dir, exist_ok=True)

# Save models
torch.save(model_stage1.state_dict(), os.path.join(output_dir, 'model_stage1_binary.pth'))
torch.save(model_stage2.state_dict(), os.path.join(output_dir, 'model_stage2_multiclass.pth'))
print("✓ Saved PyTorch models")

# Save preprocessing artifacts
with open(os.path.join(output_dir, 'scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)

with open(os.path.join(output_dir, 'feature_selector.pkl'), 'wb') as f:
    pickle.dump(feature_selector, f)

with open(os.path.join(output_dir, 'label_encoder.pkl'), 'wb') as f:
    pickle.dump(label_encoder, f)

print("✓ Saved preprocessing artifacts")

# Save selected feature names
with open(os.path.join(output_dir, 'selected_features.txt'), 'w') as f:
    for feature in selected_features:
        f.write(f"{feature}\n")

print("✓ Saved selected feature names")

# Save configuration
with open(os.path.join(output_dir, 'config.json'), 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("✓ Saved configuration")

# Save results summary
results_summary = {
    'stage1_binary': {
        'accuracy': float(results_stage1['accuracy']),
        'auroc': float(results_stage1['auroc_macro']),
        'f1_macro': float(results_stage1['macro_f1'])
    },
    'stage2_multiclass': {
        'accuracy': float(results_stage2['accuracy']),
        'auroc': float(results_stage2['auroc_macro']),
        'f1_macro': float(results_stage2['macro_f1'])
    },
    'pipeline_combined': {
        'accuracy': float(pipeline_results['accuracy']),
        'auroc': float(pipeline_results['auroc_macro']),
        'f1_macro': float(pipeline_results['macro_f1'])
    }
}

with open(os.path.join(output_dir, 'results_summary.json'), 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✓ Saved results summary")
print("="*60)
print(f"\n✅ All artifacts saved to: {output_dir}")

## 📋 Cell 23: Final Summary Report

In [24]:
print("\n" + "="*80)
print("📋 FINAL SUMMARY REPORT")
print("="*80)

print("\n🎯 PIPELINE OVERVIEW:")
print(f"   Total samples: {len(y)}")
print(f"   Total features (original): {len(feature_names)}")
print(f"   Selected features: {len(selected_features)}")
print(f"   Number of classes (all): {num_classes}")
print(f"   Number of diseases (Stage 2): {len(multiclass_names_stage2)}")
print(f"   Train/Val/Test split: {len(data_splits['y_train'])}/{len(data_splits['y_val'])}/{len(data_splits['y_test'])}")

print("\n🩺 STAGE 1 (Binary - Healthy vs Diseased):")
print(f"   Accuracy:      {results_stage1['accuracy']:.4f}")
print(f"   AUROC:         {results_stage1['auroc_macro']:.4f}")
print(f"   Macro F1:      {results_stage1['macro_f1']:.4f}")
print(f"   Recall (Diseased): {results_stage1['per_class_recall'][1]:.4f}")

print("\n🧫 STAGE 2 (Multi-Class Disease - EXCLUDES HEALTHY):")
print(f"   Number of classes: {len(multiclass_names_stage2)} diseases")
print(f"   Accuracy:      {results_stage2['accuracy']:.4f}")
print(f"   AUROC (Macro): {results_stage2['auroc_macro']:.4f}")
print(f"   AUROC (Micro): {results_stage2['auroc_micro']:.4f}")
print(f"   Macro F1:      {results_stage2['macro_f1']:.4f}")
print(f"   Micro F1:      {results_stage2['micro_f1']:.4f}")

print("\n🔗 COMBINED TWO-STAGE PIPELINE:")
print(f"   Accuracy:      {pipeline_results['accuracy']:.4f}")
print(f"   AUROC (Macro): {pipeline_results['auroc_macro']:.4f}")
print(f"   AUROC (Micro): {pipeline_results['auroc_micro']:.4f}")
print(f"   Macro F1:      {pipeline_results['macro_f1']:.4f}")
print(f"   Micro F1:      {pipeline_results['micro_f1']:.4f}")

print("\n📁 SAVED ARTIFACTS:")
print(f"   - model_stage1_binary.pth (2 classes)")
print(f"   - model_stage2_multiclass.pth ({len(multiclass_names_stage2)} disease classes)")
print(f"   - scaler.pkl")
print(f"   - feature_selector.pkl")
print(f"   - label_encoder.pkl")
print(f"   - selected_features.txt")
print(f"   - config.json")
print(f"   - results_summary.json")

print("\n" + "="*80)
print("✅ Pipeline execution complete!")
print(f"   ⚠️  IMPORTANT: Stage 2 was trained on {len(multiclass_names_stage2)} disease classes only (Healthy excluded)")
print("="*80)


📋 FINAL SUMMARY REPORT

🎯 PIPELINE OVERVIEW:
   Total samples: 14944
   Total features (original): 2378
   Selected features: 500
   Number of classes (all): 11
   Number of diseases (Stage 2): 10
   Train/Val/Test split: 10460/2242/2242

🩺 STAGE 1 (Binary - Healthy vs Diseased):
   Accuracy:      0.8301
   AUROC:         0.9122
   Macro F1:      0.8300
   Recall (Diseased): 0.8136

🧫 STAGE 2 (Multi-Class Disease - EXCLUDES HEALTHY):
   Number of classes: 10 diseases
   Accuracy:      0.7690
   AUROC (Macro): 0.9657
   AUROC (Micro): 0.9709
   Macro F1:      0.7601
   Micro F1:      0.7690

🔗 COMBINED TWO-STAGE PIPELINE:
   Accuracy:      0.7435
   AUROC (Macro): 0.9527
   AUROC (Micro): 0.9705
   Macro F1:      0.6295
   Micro F1:      0.7435

📁 SAVED ARTIFACTS:
   - model_stage1_binary.pth (2 classes)
   - model_stage2_multiclass.pth (10 disease classes)
   - scaler.pkl
   - feature_selector.pkl
   - label_encoder.pkl
   - selected_features.txt
   - config.json
   - results_summary.